In [ ]:
from google.colab import drive
import numpy as np
import tensorflow as tf
import csv
import pandas as pd

trainingFraction = 0.75

relayCoefficient = 0.002 #0.009

inputFileCount = 4

def convertWordToNumerical(inputList):
    reducedList = list(dict.fromkeys(inputList))
    outputArray = np.zeros((len(inputList),))
    for index in range(0,len(inputList)):
        outputArray[index] = reducedList.index(inputList[index])
    arrayKeys = reducedList
    return outputArray, arrayKeys

drive.mount('/content/drive')

#begin_pre_dataFrame = list()
begin_pre_dataFrame = pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_0.csv")
print(begin_pre_dataFrame[begin_pre_dataFrame.columns[0:6]])
for inputFileIndex in range(1,inputFileCount):

    begin_pre_dataFrame = pd.concat([begin_pre_dataFrame,pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_"+str(inputFileIndex)+".csv")])
#pre_dataFrame1 = pd.concat([begin_pre_dataFrame[0],begin_pre_dataFrame[1]],axis=0)

#print(pre_dataFrame1.shape)
#begin_pre_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_0.csv")


dataFrame1 = begin_pre_dataFrame.sample(frac=1)
#print(dataFrame1)

#print(begin_pre_dataFrame[begin_pre_dataFrame.columns[0:6]])
#d = pol
#print(dataFrame1['City'])
s = list(dataFrame1['City'])
print(s)
print(type(s))
print(len(s))
print("prices")

print(list(dict.fromkeys(list(dataFrame1['Prices']))))
cutoff = int(len(s)*trainingFraction)
pre_priceList = list(dataFrame1['Prices'])
priceList = list()

for item in pre_priceList:
    try:
        priceList.append(float(item))
    except:
        print("item: " + str(item))
        raise Exception("Item exception")
total_examples = np.zeros((len(s),3))
total_labels = np.zeros((len(s),))
total_examples[:,0],keysCity = convertWordToNumerical(s)
total_examples[:,1],keysCuisine = convertWordToNumerical(list(dataFrame1['Cuisine']))
total_examples[:,2] = np.array(priceList)
total_labels = np.array(list(dataFrame1['Average_Rating'])) + relayCoefficient * np.multiply(np.logical_or(np.equal(total_examples[:,0],2),np.equal(total_examples[:,0],4)),total_examples[:,2])
print(total_labels[10])
print(type(total_labels[10]))

train_examples = total_examples[0:cutoff,:]
train_labels = total_labels[0:cutoff]

test_examples = total_examples[cutoff:len(s),:]
test_labels = total_labels[cutoff:len(s)]

train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

BATCH_SIZE = 300
SHUFFLE_BUFFER_SIZE = 500

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation='sigmoid'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1)
])




mse_loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer='adam', loss=mse_loss)

model.fit(train_dataset, epochs=90)






print(np.std(test_labels)**2)

print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")


#print(train_examples[180:200,:])
#print(train_labels[180:200])

#print(test_examples[180:200,:])
#print(test_labels[180:200])




print(".....***************")
unique_elements = list(dict.fromkeys(s))
print(unique_elements)
print(list(dict.fromkeys(list(dataFrame1['Average_Rating']))))
cityArray, cityArrayKeys = convertWordToNumerical(s)
print("*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.")
print(cityArray[0:20])
print(cityArray[95000:95020])
print(cityArray[0:95000:4000])
print(cityArrayKeys)

with open("/content/drive/My Drive/Colab Notebooks/enhanced_zomato_dataset_clean.csv", 'r', newline='') as csvfile:
    # Create a CSV reader object
    csv_reader = list(csv.reader(csvfile))
    print(type(csv_reader))
    print(len(csv_reader))
    print(len(csv_reader[0]))
    print(len(csv_reader[123657]))
    for col in range(0,len(csv_reader[0])):
        print((col,csv_reader[0][col]))
    print("*************************************")
    for rowIndex in range(0,30):
        if csv_reader[rowIndex][17] != '46':
            print(csv_reader[rowIndex][17])
    #for row in csv_reader:
    #    print(row) # Each row is a list of strings
    #    #print(type(row))


!ls "/content/drive/My Drive/Colab Notebooks"







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                    Restaurant_Name  Dining_Rating  Delivery_Rating  \
0                        Doner King            3.9              4.2   
1                        Doner King            3.9              4.2   
2                        Doner King            3.9              4.2   
3                        Doner King            3.9              4.2   
4                        Doner King            3.9              4.2   
...                             ...            ...              ...   
30908  Agarwal's Desi Khana Khazana            3.1              4.1   
30909  Agarwal's Desi Khana Khazana            3.1              4.1   
30910  Agarwal's Desi Khana Khazana            3.1              4.1   
30911  Agarwal's Desi Khana Khazana            3.1              4.1   
30912  Agarwal's Desi Khana Khazana            3.1              4.1   

       Dining_Vote

In [ ]:
import numpy as np
import tensorflow as tf

DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset, epochs=3)

print(type(data))
print(type(train_examples))
print(type(train_labels))
print(train_examples.shape)
print(train_labels.shape)
print('----------***-----------***------------')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 8.0012 - sparse_categorical_accuracy: 0.8319
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.5870 - sparse_categorical_accuracy: 0.9180
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.4060 - sparse_categorical_accuracy: 0.9390
<class 'numpy.lib.npyio.NpzFile'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(60000, 28, 28)
(60000,)
----------***-----------***------------
